In [56]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import r2_score


In [57]:
X,y=load_diabetes(return_X_y=True)

In [58]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [59]:
x_train # scaling pehla hi hoyi payi aa so we dont need to do again

array([[ 0.07076875,  0.05068012,  0.01211685, ...,  0.03430886,
         0.02736405, -0.0010777 ],
       [-0.00914709,  0.05068012, -0.01806189, ...,  0.07120998,
         0.00027248,  0.01963284],
       [ 0.00538306, -0.04464164,  0.04984027, ..., -0.00259226,
         0.01703607, -0.01350402],
       ...,
       [ 0.03081083, -0.04464164, -0.02021751, ..., -0.03949338,
        -0.01090325, -0.0010777 ],
       [-0.01277963, -0.04464164, -0.02345095, ..., -0.00259226,
        -0.03845972, -0.03835666],
       [-0.09269548, -0.04464164,  0.02828403, ..., -0.03949338,
        -0.00514219, -0.0010777 ]])

In [60]:
# first of all lets go with simple model
lr=LinearRegression()
lr.fit(x_train,y_train)
y_pred=lr.predict(x_test)
r2_score(y_test,y_pred)

0.4526027629719196

In [61]:
lr.coef_,lr.intercept_

(array([  37.90402135, -241.96436231,  542.42875852,  347.70384391,
        -931.48884588,  518.06227698,  163.41998299,  275.31790158,
         736.1988589 ,   48.67065743]),
 np.float64(151.34560453985995))

In [62]:
# go for more models for regularization which reduce overfitting
rd=Ridge(alpha=1.0) # alpha is 1 for ridge because its weak so in order to have an impact of this we have to keep the alpha=1
rd.fit(x_train,y_train)
y_pred=rd.predict(x_test)
r2_score(y_test,y_pred)

0.41915292635986545

In [63]:
ls=Lasso(alpha=0.1) # alpha is 0.1 for lasso because its aggresive
ls.fit(x_train,y_train)
y_pred=ls.predict(x_test)
r2_score(y_test,y_pred)

0.4718547867276227

In [64]:
import random
class mbgdregressor:
    def __init__(self,batch_size=50,learning_rate=0.1,epochs=100):
        self.coef_=None
        self.intercept_=None
        self.learning_rate=learning_rate
        self.epochs=epochs
        self.batch_size=batch_size
    
    def fit(self,x_train,y_train):
        # initiate your coef and intercept
        self.coef_=np.ones(x_train.shape[1])
        self.intercept_=0
        # update coef and intercept
        for i in range(self.epochs):
            for j in range(int(x_train.shape[0] / self.batch_size)):
                idx=random.sample(range(1,x_train.shape[0]),self.batch_size)
                y_hat=np.dot(x_train[idx],self.coef_)+self.intercept_
                intercept_der= -2 * np.mean(y_train[idx] - y_hat)
                self.intercept_=self.intercept_-(self.learning_rate*intercept_der)

                coef_der=-2 * np.dot((y_train[idx] - y_hat),x_train[idx])
                self.coef_=self.coef_ - (self.learning_rate * coef_der)
        print(self.intercept_,self.coef_)
    def predict(self,x_test):
        return np.dot(x_test,self.coef_)+self.intercept_

In [65]:
# just to show working 
import random
random.sample(range(1,100),10)   # from 1 to 100 we need 10 random numbers

[13, 70, 26, 79, 41, 51, 86, 32, 48, 56]

In [66]:
gd=mbgdregressor()
gd.fit(x_train,y_train)
y_pred_1=gd.predict(x_test)
r2_score(y_test,y_pred)


155.19297431593395 [  75.30344155 -217.96197953  542.01289975  361.94394003 -170.31124215
  -32.53258299 -185.32233405  170.3759895   435.14467566   66.66469117]


0.4718547867276227

**sklearn does not have the direct class of mini batch gd so we have to apply some with the clever trick**

In [67]:
from sklearn.linear_model import SGDRegressor

In [68]:
sgd=SGDRegressor(learning_rate='constant',eta0=0.1)

In [69]:
batch_size=35
for i in range(100):
    idx=random.sample(range(0,x_train.shape[0]),batch_size)
    sgd.partial_fit(x_train[idx],y_train[idx]) # only 35 rows are passed 

In [70]:
sgd.coef_,sgd.intercept_

(array([  56.26623516,  -95.60582195,  366.82817717,  234.27542786,
          -3.08888928,  -41.22565732, -179.17392505,  150.19483326,
         277.04623001,  153.31277654]),
 array([168.0932263]))

In [72]:
y_pred_2=sgd.predict(x_test)

In [73]:
r2_score(y_test,y_pred_2)

0.4050805922595242